## Prepare & clean the data

In [1]:
# Extract error types from: "conll14st-preprocessed.conll.ann" file
# Separate each word of a particular error type in its own list.
l1=[]
l4=[]
l5=[]
l6=[]
l7=[]
l11=[]
l14=[]
def extract_err_type(colnn_ann):
    f = open(colnn_ann, 'r')
    for line in f:
        parts = line.split()
        if(len(parts)>0):
            if(line[0:6]=="<TYPE>"):
                type_ = line[6:len(parts[0])-7]
            elif(line[0:12]=="<CORRECTION>"):
                corrected_word = line[12:len(line)-14] 
                if type_ =="Vt":
                    l1.append(line[12:len(line)-14])   
                if type_ =="Vform":
                    l4.append(line[12:len(line)-14])
                if type_ =="SVA":
                    l5.append(line[12:len(line)-14])
                if type_ =="ArtOrDet":
                    l6.append(line[12:len(line)-14])
                if type_ =="Nn":
                    l7.append(line[12:len(line)-14])
                if type_ =="Prep":
                    l11.append(line[12:len(line)-14])
                if type_ =="Wform":
                    l14.append(line[12:len(line)-14])
    return "Done"

In [2]:
# Pass in to the function, the path to conll14st-preprocessed.conll.ann in your system
extract_err_type("/Users/highsierra/Tech-Skills/Labortory/ML-Fundamentals/error_types_conll14/conll14st-preprocessed.conll.ann")

'Done'

In [3]:
# Create a corresponding list containing the error type of each word
# Map between each word and its error type
ll1=[]
for i in range(len(l1)):
    ll1.append('Vt')

ll4=[]
for i in range(len(l4)):
    ll4.append('Vform')

ll5=[]
for i in range(len(l5)):
    ll5.append('SVA')

ll6=[]
for i in range(len(l6)):
    ll6.append('ArtOrDet')

ll7=[]
for i in range(len(l7)):
    ll7.append('Nn')

ll11=[]
for i in range(len(l11)):
    ll11.append('Prep')


ll14=[]
for i in range(len(l14)):
    ll14.append('Wform')

In [4]:
# Separate the words (input) from the error types (target)
import itertools

targets = list(itertools.chain(ll1, ll4, ll5, ll6, ll7, ll11, ll14))
words = list(itertools.chain(l1, l4, l5, l6, l7, l11, l14))

####  P.S. Up to this point, only the representations of words have been used. No further feature engineering has been done.

#### This separates sentences from annotations without attention to the error type

import re
sen = []
errSen = []


def extract_err_sen(colnn_ann):
    f = open(colnn_ann, 'r')
    for line in f:
        sen.append(line)

    for i in range(len(sen)):
        if sen[i][0] == 'A' and sen[i-1][0] == 'S':
            errSen.append(sen[i-1])
    return "Done"


extract_err_sen("/Users/highsierra/conll14st-preprocessed.m2")


documents = []
for sen in range(0, len(errSen)):
    # Remove all the special characters
    document = re.sub(r"[^a-zA-Z/'/]", ' ', str(errSen[sen]))

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'S' and ending with 'n'
    document = re.sub(r"^S", '', document)
    document = re.sub(r"\.\n", '', document)

    # Removing trailling space
    document = re.sub(r'^\s', '', document, flags=re.I)
    document = re.sub(r'\s$', '', document, flags=re.I)

    documents.append(document)
    
print(len(documents))
print(documents[0])

texts = []
texts = documents[0:21138]
print(len(texts))


In [5]:
import re

with open("/Users/highsierra/conll14st-preprocessed.m2") as f:
    lines = f.read()
tests = lines.split("\n\n")

documents = []
doc = []

for sen in range(0, len(tests)):
    if re.search("Vt", str(tests[sen])) or re.search("Vform", str(tests[sen])) or re.search("SVA", str(tests[sen])) or re.search("ArtOrDet", str(tests[sen])) or re.search("Nn", str(tests[sen])) or re.search("Prep", str(tests[sen])) or re.search("Wform", str(tests[sen])):
        documents.append(str(tests[sen]))
        doc += str(tests[sen]).split(".\n")

cln = []
for k in range(0, len(doc)):
    if doc[k][0] == 'S':
        cln.append(doc[k])

errSen = []
for sen in range(0, len(cln)):
    # Remove all the special characters
    err = re.sub(r"[^a-zA-Z/'/]", ' ', str(cln[sen]))

    # Substituting multiple spaces with single space
    err = re.sub(r'\s+', ' ', err, flags=re.I)

    # Removing prefixed 'S' and ending with 'n'
    err = re.sub(r"^S", '', err)
    err = re.sub(r"\.\n", '', err)

    # Removing trailling space
    err = re.sub(r'^\s', '', err, flags=re.I)
    err = re.sub(r'\s$', '', err, flags=re.I)

    errSen.append(err)
print(errSen[1])
print(len(errSen))


The solution can be obtain by using technology to achieve a better usage of space that we have and resolve the problems in lands that inhospitable such as desserts and swamps
14046


In [6]:
import spacy
import numpy as np

nlp = spacy.load("en_core_web_sm")
l1=[]
l2=[]

for text in errSen:
        doc = nlp(text)
        l1 = list([token.text, token.pos_, token.tag_, token.dep_] for token in doc if token.text in words)
        l2.extend(l1)
        arr = np.array(l2)

print(arr.shape)

(244814, 4)


## Encode the data

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(), [0, 1, 2, 3])], remainder='passthrough')

arr = transformer.fit_transform(arr)
print(arr.toarray())
print(arr.toarray().shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(244814, 2553)


In [8]:
print(len(words))

21138


In [9]:
# Encode the input using glove embeddings   

from zeugma.embeddings import EmbeddingTransformer

glove = EmbeddingTransformer('glove') 
X = glove.transform(words)

Using TensorFlow backend.


In [10]:
print(X)
print(X.shape)

[[ 9.4602001e-04 -8.4354997e-02  1.9994000e-01 ...  2.7858001e-01
   2.0168000e-01 -7.1859002e-01]
 [ 5.7466000e-02  1.7890000e-01 -6.2778002e-01 ... -7.5557001e-02
   2.4191999e-01  2.2470001e-03]
 [-5.2309000e-01 -6.8026000e-01 -4.6072000e-01 ...  5.2221000e-01
   7.0643997e-01 -2.5231999e-01]
 ...
 [ 5.6629997e-01  2.5286999e-01 -1.1373000e+00 ...  5.4163003e-01
  -1.7024000e+00 -4.1539001e-01]
 [ 3.4362650e-01  9.2356995e-02 -6.9733500e-01 ... -3.7974998e-02
  -1.1478500e+00 -2.5225499e-01]
 [ 1.0357000e-01  5.4285997e-01 -8.8192999e-01 ... -4.9219000e-01
   1.3172001e-01 -8.6390001e-01]]
(21138, 25)


In [11]:
print(len(targets))

21138


In [12]:
print(arr.toarray().dtype)
print(np.sum(arr))

float64
979256.0


In [13]:
arr2d= np.reshape(arr, (6, -1))
print(arr2d.toarray())
print(arr2d.shape)
print(np.sum(arr2d)) 

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(6, 104168357)
979256.0


In [14]:
wall = np.zeros((6,104171587),dtype=np.float64) 

In [15]:
print(wall)
print(np.sum(wall))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.0


In [16]:
wall[:, 0:104168357] += arr2d

In [17]:
print(wall.shape)
print(np.sum(wall))

(6, 104171587)
979256.0


In [18]:
wall2d= np.reshape(wall, (21138,-1))

In [19]:
print(wall2d.shape)
print(np.sum(wall2d))

(21138, 29569)
979256.0


In [20]:
print("concatenate column wise")
concat = np.concatenate((X,wall2d),axis=1)
print(concat)

concatenate column wise
[[ 9.46020009e-04 -8.43549967e-02  1.99939996e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 5.74660003e-02  1.78900003e-01 -6.27780020e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-5.23090005e-01 -6.80260003e-01 -4.60720003e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 5.66299975e-01  2.52869993e-01 -1.13730001e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 3.43626499e-01  9.23569947e-02 -6.97335005e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.03569999e-01  5.42859972e-01 -8.81929994e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [21]:
print(concat.shape)

(21138, 29594)


In [22]:
# Encode the target vector in binary 

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
ybin = lb.fit(targets)
lb.classes_

array(['ArtOrDet', 'Nn', 'Prep', 'SVA', 'Vform', 'Vt', 'Wform'],
      dtype='<U8')

In [23]:
y = lb.transform(targets)
print(y)
print(y.shape)

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]
(21138, 7)


## Create training & testing sets

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(concat, y, test_size=0.2, random_state=0)

## Train the model

In [28]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(300,200,100), max_iter=500, activation = 'relu', solver='adam', random_state=21, tol=0.000000001)
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300, 200, 100), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=21, shuffle=True, solver='adam',
              tol=1e-09, validation_fraction=0.1, verbose=False,
              warm_start=False)

## Make predictions

In [26]:
y_pred = clf.predict(X_test)

## Evaluate the model

In [27]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


print(classification_report(y_test,y_pred,target_names=lb.classes_))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

    ArtOrDet       0.73      0.76      0.74      1319
          Nn       0.64      0.49      0.55       760
        Prep       0.64      0.40      0.49       496
         SVA       0.27      0.18      0.22       300
       Vform       0.17      0.06      0.09       266
          Vt       0.53      0.37      0.44       654
       Wform       0.26      0.29      0.27       433

   micro avg       0.57      0.47      0.52      4228
   macro avg       0.46      0.36      0.40      4228
weighted avg       0.56      0.47      0.51      4228
 samples avg       0.47      0.47      0.47      4228

0.46759697256385996


/opt/miniconda3/envs/tensorflow1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
